# [E - 14] Movielens 영화 추천 실습

이번에 활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터입니다.
- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다
- 별점을 시청횟수로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

## 1) 라이브러리와 데이터 준비

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## 2) 데이터 전처리

유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하므로 3점 이상만 남긴다.

In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


ratings 즉 별점을 시청횟수로 생각하기 때문에 counts로 변환

In [3]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [4]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


### ratings 데이터프레임과 movies 데이터프레임 합치기

In [6]:
ratings = ratings.join(movies.set_index('movie_id'), on='movie_id')
ratings.head()

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


### 필요없는 열 timestamp와 genre 삭제

In [7]:
ratings.drop(['timestamp','genre'], axis = 1 , inplace= True )
ratings

,user_id,movie_id,counts,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"
...,...,...,...,...
1000203,6040,1090,3,Platoon (1986)
1000205,6040,1094,5,"Crying Game, The (1992)"
1000206,6040,562,5,Welcome to the Dollhouse (1995)
1000207,6040,1096,4,Sophie's Choice (1982)


## 3) 데이터 탐색

- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [8]:
print('영화 개수: ', ratings['movie_id'].nunique())
print('사용자 수: ', ratings['user_id'].nunique())

영화 개수:  3628
사용자 수:  6039


In [9]:
# 가장 인기 있는 영화 30개(인기순)
movie_title_count = ratings.groupby('title')['user_id'].count()
movie_title_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

### 검색하기 쉽게 영화 제목들을 소문자로 변경

In [10]:
ratings['title'] = ratings['title'].str.lower()
ratings.head()

,user_id,movie_id,counts,title
0,1,1193,5,one flew over the cuckoo's nest (1975)
1,1,661,3,james and the giant peach (1996)
2,1,914,3,my fair lady (1964)
3,1,3408,4,erin brockovich (2000)
4,1,2355,5,"bug's life, a (1998)"


## 4) 내가 선호하는 영화 5가지 추가

In [11]:
movies[movies['title'].str.lower().str.contains('lion', regex=False)]

,movie_id,title,genre
360,364,"Lion King, The (1994)",Animation|Children's|Musical
1980,2049,"Happiest Millionaire, The (1967)",Comedy|Musical


In [12]:
movies[movies['title'].str.lower().str.contains('titanic', regex=False)]

,movie_id,title,genre
1672,1721,Titanic (1997),Drama|Romance
2088,2157,"Chambermaid on the Titanic, The (1998)",Romance
3334,3403,Raise the Titanic (1980),Drama|Thriller
3335,3404,Titanic (1953),Action|Drama


In [13]:
movies[movies['title'].str.lower().str.contains('peter', regex=False)]

,movie_id,title,genre
2018,2087,Peter Pan (1953),Animation|Children's|Fantasy|Musical
2976,3045,Peter's Friends (1992),Comedy|Drama


In [14]:
movies[movies['title'].str.lower().str.contains('terminator', regex=False)]

,movie_id,title,genre
585,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
1220,1240,"Terminator, The (1984)",Action|Sci-Fi|Thriller


In [15]:
my_favorite = ['Lion King, The (1994)','Titanic (1997)','Peter Pan (1953)','Terminator 2: Judgment Day (1991)','Terminator, The (1984)']
my_favorite

['Lion King, The (1994)',
 'Titanic (1997)',
 'Peter Pan (1953)',
 'Terminator 2: Judgment Day (1991)',
 'Terminator, The (1984)']

In [16]:
for i in range(len(my_favorite)):
    my_favorite[i] = my_favorite[i].lower()
my_favorite

['lion king, the (1994)',
 'titanic (1997)',
 'peter pan (1953)',
 'terminator 2: judgment day (1991)',
 'terminator, the (1984)']

In [17]:
my_favorite_id = [364,1721,2087,589,1240]

In [18]:
my_movie_list = pd.DataFrame({'user_id': ['hobin']*5, 'movie_id': my_favorite_id ,'title': my_favorite, 'counts': [5]*5 })

if not ratings.isin({'user_id': ['hobin']})['user_id'].any():
    ratings = ratings.append(my_movie_list)
    
ratings.tail(10)

,user_id,movie_id,counts,title
1000203,6040,1090,3,platoon (1986)
1000205,6040,1094,5,"crying game, the (1992)"
1000206,6040,562,5,welcome to the dollhouse (1995)
1000207,6040,1096,4,sophie's choice (1982)
1000208,6040,1097,4,e.t. the extra-terrestrial (1982)
0,hobin,364,5,"lion king, the (1994)"
1,hobin,1721,5,titanic (1997)
2,hobin,2087,5,peter pan (1953)
3,hobin,589,5,terminator 2: judgment day (1991)
4,hobin,1240,5,"terminator, the (1984)"


### 모델에 활용하기 위한 전처리

#### indexing

In [19]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
title_unique = ratings['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [20]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['hobin'])    # 6040명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(title_to_idx['lion king, the (1994)'])

6039
330


In [21]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings): # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')
# artist_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다.
temp_movie_data = ratings['title'].map(title_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('title column indexing OK!!')
    ratings['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

ratings


user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,counts,title
0,0,1193,5,0
1,0,661,3,1
2,0,914,3,2
3,0,3408,4,3
4,0,2355,5,4
...,...,...,...,...
0,6039,364,5,330
1,6039,1721,5,27
2,6039,2087,5,551
3,6039,589,5,92


## 5) CSR matrix 만들기

In [22]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_title = ratings['title'].nunique()

csr_data = csr_matrix((ratings['counts'], (ratings.user_id, ratings.title)), shape= (num_user, num_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 6) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

AlternatingLeastSquares 클래스의 __init__ 파라미터를 살펴보겠습니다.

- factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
- regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
- use_gpu : GPU를 사용할 것인지
- iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

factors 와 iterations를 늘릴수록 학습 데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야한다.

- implicit 패키지는 이전 스텝에서 설명한 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지입니다.  
- 이 패키지에 구현된 als(AlternatingLeastSquares) 모델을 사용하겠습니다. Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식이 효과적인 것으로 알려져 있습니다.

In [23]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

### 1. factors = 100, iterations = 15

In [24]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [25]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [26]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

- hobin 벡터와 라이온킹의 벡터를 어떻게 만들고 있는지
- 두 벡터를 곱하면 어떤 값이 나오는지

In [27]:
hobin, lion_king = user_to_idx['hobin'], title_to_idx['lion king, the (1994)']
hobin_vector, lion_king_vector = als_model.user_factors[hobin], als_model.item_factors[lion_king]

In [28]:
hobin_vector

array([-0.55472255,  0.45733714, -0.4279555 ,  0.1718062 ,  0.49608222,
       -0.7590066 ,  0.48033246,  0.7990463 ,  0.780116  , -0.6215163 ,
        0.060273  ,  0.24206685, -0.07247522,  0.36215535, -0.2894614 ,
        0.07764348, -0.20734382, -0.22625656,  0.05778775,  0.04374462,
        1.0203581 , -0.7538405 , -0.07972892,  0.26529273,  0.13792379,
        0.15132159, -0.52820575,  0.21021894,  0.02548573,  0.08505978,
       -0.21448067,  0.12688291, -0.2290185 , -0.39569232,  0.01118926,
       -1.271476  , -0.2775971 ,  0.46348676,  1.3073747 , -0.4777066 ,
       -1.3555387 ,  0.12620948, -0.5438191 , -0.2707347 ,  0.16263807,
        0.05516229, -0.10755005, -0.29717636,  0.3673155 ,  0.35799718,
       -0.45930126,  0.6832188 , -0.37656575, -0.61012316, -0.30163845,
        0.6640055 ,  0.15188597,  0.331414  , -0.42780003,  0.74579847,
        0.6747533 ,  0.7066031 ,  0.1594745 , -0.01694552, -0.2043349 ,
        0.78533953, -0.72479856,  0.57942444,  0.19184765,  0.05

In [29]:
lion_king_vector

array([-0.00161203,  0.02215509,  0.00852306,  0.03692856,  0.01443036,
       -0.03215155, -0.00688052,  0.03407206,  0.01249324, -0.00408475,
        0.01096072, -0.00133426,  0.01226388,  0.03280609, -0.01621023,
        0.02491633,  0.00895984,  0.00389363,  0.02484238,  0.01942519,
        0.03192723, -0.00220057,  0.0147907 ,  0.00183811,  0.01336761,
       -0.02951899,  0.00189081,  0.00697927, -0.02416416, -0.0108061 ,
       -0.00829271, -0.00661916, -0.00430843, -0.00019967,  0.0050997 ,
       -0.02144758,  0.01289811,  0.02710608,  0.02379449,  0.00643225,
        0.00859246,  0.01464012, -0.01291889,  0.00596846,  0.0052227 ,
       -0.01202434, -0.01870783,  0.0281225 ,  0.03374984,  0.01292805,
       -0.01229784,  0.01794098, -0.0167413 ,  0.00063757, -0.00455761,
        0.00490464,  0.03805896,  0.00180386, -0.02298776,  0.01966003,
        0.00607101,  0.02009414,  0.01011961,  0.00431272,  0.01162998,
        0.02708894, -0.01024291,  0.00611023,  0.01279618,  0.02

In [30]:
# hobin과 lion_king를 내적하는 코드
np.dot(hobin_vector,lion_king_vector)

0.45943338

### 내가 선호하지 않은 영화 선호도

In [31]:
american_beauty = title_to_idx['american beauty (1999)']
american_beauty_vector = als_model.item_factors[american_beauty]
np.dot(hobin_vector, american_beauty_vector)

0.0752985

### 2. factors = 150, iterations = 20

In [32]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=150, regularization=0.01, use_gpu=False, iterations=20, dtype=np.float32)

In [33]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [34]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/20 [00:00<?, ?it/s]

- hobin 벡터와 라이온킹의 벡터를 어떻게 만들고 있는지
- 두 벡터를 곱하면 어떤 값이 나오는지

In [35]:
hobin, lion_king = user_to_idx['hobin'], title_to_idx['lion king, the (1994)']
hobin_vector, lion_king_vector = als_model.user_factors[hobin], als_model.item_factors[lion_king]

In [36]:
hobin_vector

array([-0.23955983,  0.47754848, -1.1354696 ,  0.16628672,  0.2832987 ,
        0.37694788,  0.83535093,  0.26804426, -0.5613626 , -0.13661169,
        0.83457583,  0.80587745,  0.44893605, -0.16617349, -0.5013347 ,
        0.06992405, -0.14950217, -0.34706578, -1.042501  ,  0.38847554,
       -0.826115  ,  0.01129321, -0.6245959 , -0.1764752 , -0.23510493,
       -0.47813502,  0.2743627 , -0.05217784,  0.14295706,  0.5244392 ,
        0.2789682 ,  0.24556682, -0.15414062, -0.1655981 , -0.2164505 ,
       -0.08545314, -0.618842  , -0.07637605, -0.11097969,  0.23718487,
        1.0678276 ,  0.4247299 , -0.41123345,  0.32682395,  0.15715909,
        0.01824251, -0.36569333,  0.74155784,  0.58693683,  0.3695818 ,
       -0.19213967,  0.11727358, -0.14973821, -0.17185017,  0.92226607,
        0.07887599, -0.41778204,  0.2939218 , -0.11222482,  0.0943589 ,
        0.67799747,  0.03904625, -0.00904965, -0.05493079, -0.5715517 ,
        0.11893825,  0.51459885, -0.16071175,  0.32380655, -0.20

In [37]:
lion_king_vector

array([ 1.06168874e-02,  1.75339319e-02, -8.09989404e-03,  2.89467610e-02,
       -2.10711529e-04,  2.27353051e-02,  9.83200502e-03, -3.68753634e-03,
       -3.05939466e-02,  1.01456763e-02,  6.82614045e-03,  2.83115404e-03,
        4.21444187e-03,  1.45475473e-03,  5.36311790e-03, -1.51209198e-02,
       -5.69259457e-04,  1.83353468e-03, -7.31501635e-03,  9.30357259e-03,
       -2.58432534e-02, -1.12522757e-02, -2.61068214e-02,  1.98905735e-04,
       -1.35764815e-02,  1.96893830e-02,  9.64778941e-03, -1.51510853e-02,
       -1.34167017e-03,  3.11765671e-02,  5.38829993e-03,  1.97290201e-02,
       -5.09448862e-03,  2.92529669e-02,  8.10951833e-03, -3.09374672e-03,
       -3.05831488e-02,  1.91823803e-02, -1.12227993e-02,  3.09517104e-02,
        2.39442717e-02,  3.32973897e-02,  6.42406009e-03,  2.02336814e-02,
       -8.69177189e-03,  2.06959564e-02, -1.14952419e-02,  3.29256337e-03,
        1.42726051e-02,  5.37508354e-03,  1.10868486e-02, -4.69986256e-03,
        8.80367309e-03,  

In [38]:
# hobin과 lion_king를 내적하는 코드
np.dot(hobin_vector,lion_king_vector)

0.51750183

### 내가 선호하지 않은 영화 선호도

In [39]:
american_beauty = title_to_idx['american beauty (1999)']
american_beauty_vector = als_model.item_factors[american_beauty]
np.dot(hobin_vector, american_beauty_vector)

-0.048276413

### 3. factors = 200, iterations = 30

In [40]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=200, regularization=0.01, use_gpu=False, iterations=30, dtype=np.float32)

In [41]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [42]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/30 [00:00<?, ?it/s]

- hobin 벡터와 라이온킹의 벡터를 어떻게 만들고 있는지
- 두 벡터를 곱하면 어떤 값이 나오는지

In [43]:
hobin, lion_king = user_to_idx['hobin'], title_to_idx['lion king, the (1994)']
hobin_vector, lion_king_vector = als_model.user_factors[hobin], als_model.item_factors[lion_king]

In [44]:
hobin_vector

array([ 0.05554899,  0.3645162 , -0.14574447, -0.32353082, -0.22610828,
       -0.25146782,  0.2809954 , -0.51348925, -0.27409464,  0.29799348,
       -0.02910691,  0.18648876,  0.02334903, -0.06709758, -0.41714337,
       -0.97314304, -0.79069585,  0.23593485,  0.1147014 , -0.13290809,
        0.20091644,  0.44387963,  0.6800429 ,  0.18645178, -0.3524605 ,
       -0.39997765,  0.226996  , -0.4002479 ,  0.15000425, -0.06574223,
        0.48543987,  0.38686007, -0.3122691 ,  0.2171271 ,  0.576458  ,
        0.62514406,  0.05924992, -0.2386114 ,  0.24721025, -0.01153212,
       -0.09728526, -0.19819105,  0.15390769, -0.15259014,  0.58941436,
        0.20947963,  0.14895763, -0.08134786, -0.3078983 , -0.16195111,
        0.30493635, -0.602417  , -0.24489172,  0.6611892 ,  0.12210327,
       -0.31097016,  0.76262873, -0.46201447, -0.2121455 , -0.24555884,
        0.520442  , -0.4125938 , -0.2911986 , -0.08597346, -0.29136893,
       -0.2695653 ,  0.01516703, -0.07082234, -0.2872676 , -0.11

In [45]:
lion_king_vector

array([ 0.00320075,  0.00398505,  0.00103908,  0.00412419,  0.0037981 ,
       -0.00197777,  0.02719373, -0.01735177, -0.00928479,  0.03318682,
        0.01316135, -0.00632572,  0.00172531,  0.00069103,  0.00630551,
       -0.0090883 , -0.00280754,  0.02073678, -0.00018513,  0.04011541,
       -0.00383117,  0.0114989 ,  0.01273317,  0.02314411,  0.00081946,
        0.01669025,  0.02354991, -0.01514659,  0.00798282,  0.00056663,
        0.02599023,  0.01678187,  0.00518211,  0.02076403,  0.00959884,
        0.0081941 ,  0.00835491,  0.0077666 , -0.00571992, -0.01237257,
        0.024343  ,  0.01798096,  0.00069599, -0.02357939,  0.01500432,
        0.01137678,  0.00246768, -0.01171164, -0.02261705, -0.01512164,
        0.01524682,  0.00403986,  0.00669145,  0.00489102,  0.00024124,
       -0.00436171,  0.04819167,  0.010043  , -0.00242692,  0.00192307,
        0.03099991, -0.03227696,  0.01643553,  0.00927514, -0.00406159,
        0.0013751 , -0.00065084,  0.00534812,  0.02448274,  0.00

In [46]:
# hobin과 lion_king를 내적하는 코드
np.dot(hobin_vector,lion_king_vector)

0.5696552

### 내가 선호하지 않은 영화 선호도

In [47]:
american_beauty = title_to_idx['american beauty (1999)']
american_beauty_vector = als_model.item_factors[american_beauty]
np.dot(hobin_vector, american_beauty_vector)

-0.032157067

## 7) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악

| |선호하는 영화 선호도|선호하지 않는 영화 선호도|
|----|------|---|
|100,15|0.46|0.08|
|150,20|0.52|0.05|
|200,30|0.57|-0.32|

## 8) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [49]:
favorite_movie = 'lion king, the (1994)'
movie_id = title_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie 

[(330, 0.9999999),
 (10, 0.7155579),
 (33, 0.66811115),
 (35, 0.5001754),
 (34, 0.49506488),
 (329, 0.4777461),
 (30, 0.45349503),
 (32, 0.4494569),
 (191, 0.44415727),
 (16, 0.43798313),
 (37, 0.4233991),
 (25, 0.39437878),
 (8, 0.39351866),
 (46, 0.38764527),
 (40, 0.3412886)]

In [50]:
idx_to_movie = {v:k for k,v in title_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['lion king, the (1994)',
 'beauty and the beast (1991)',
 'aladdin (1992)',
 'hunchback of notre dame, the (1996)',
 'mulan (1998)',
 'anastasia (1997)',
 'antz (1998)',
 'hercules (1997)',
 'little mermaid, the (1989)',
 'tarzan (1999)',
 'cinderella (1950)',
 'pocahontas (1995)',
 'snow white and the seven dwarfs (1937)',
 'dumbo (1941)',
 'toy story (1995)']

In [51]:
def get_similar_movie(movie_title: str):
    movie_id = title_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id, N=15)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [52]:
get_similar_movie('titanic (1997)')

['titanic (1997)',
 'ever after: a cinderella story (1998)',
 "you've got mail (1998)",
 'ghost (1990)',
 'jerry maguire (1996)',
 'walking dead, the (1995)',
 'piano, the (1993)',
 'fried green tomatoes (1991)',
 'pump up the volume (1990)',
 'city of angels (1998)',
 'far and away (1992)',
 'gay deceivers, the (1969)',
 'train of life (train de vie) (1998)',
 'dirty dancing (1987)',
 'paris, france (1993)']

## 9) 내가 가장 좋아할 만한 영화들을 추천

In [53]:
user = user_to_idx['hobin']

movie_recommended = als_model.recommend(user, csr_data, N=15, filter_already_liked_items=True)
movie_recommended

[(124, 0.45856565),
 (10, 0.35195404),
 (651, 0.3346375),
 (8, 0.30662674),
 (33, 0.2980238),
 (37, 0.2919738),
 (191, 0.24970613),
 (62, 0.24809846),
 (547, 0.22570777),
 (194, 0.22352067),
 (46, 0.22272587),
 (236, 0.22044423),
 (107, 0.20257254),
 (34, 0.20234816),
 (572, 0.20031679)]

In [54]:
[idx_to_movie[i[0]] for i in movie_recommended]

['matrix, the (1999)',
 'beauty and the beast (1991)',
 'aliens (1986)',
 'snow white and the seven dwarfs (1937)',
 'aladdin (1992)',
 'cinderella (1950)',
 'little mermaid, the (1989)',
 'total recall (1990)',
 'lady and the tramp (1955)',
 'die hard (1988)',
 'dumbo (1941)',
 'speed (1994)',
 'jurassic park (1993)',
 'mulan (1998)',
 'sleeping beauty (1959)']

### 영화들을 추천하기 위힌 내 선호 영화들의 기여도

In [55]:
recommended = title_to_idx['matrix, the (1999)']
explain = als_model.explain(user, csr_data, itemid=recommended)

[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('terminator 2: judgment day (1991)', 0.3143127330417084),
 ('terminator, the (1984)', 0.10182598830151213),
 ('peter pan (1953)', 0.015631110894192224),
 ('titanic (1997)', 0.014244244832544823),
 ('lion king, the (1994)', 0.007715538110634232)]

터미네이터 1,2로 인해 메트릭스를 추천해줬다.

## 회고

- 가장 아쉬웠던 점은 영화 데이터들이 년도를 보면 1900년도대의 영화들이 대부분이라 정말 유명한 영화가 아니면 모른다는 것이다. 그래서 이 데이터셋 안에 있는 영화들로 선호하는 영화를 선택할 때에 어려움이 있었다.
- 이 프로젝트를 진행하면서 데이터프레임을 합치는 과정이 있어 함수 join과 수 많은 데이터들 속에서 포함하는 문자열을 찾기 위한 cotain 함수를 더욱 알게되고 공부하는 시간이였다.
- 선호도를 높이기 위해 factors는 50씩 iterations는 5, 10 증가로 값을 증가시켰지만 선호도를 높이는데에 성공하였고 더욱 값을 높인다면 과적합 발생 위험때문에 3번의 테스트를 진행하였다. 